In [1]:
#import 
import os
import logging
import time
import csv
import html
import jinja2
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry
import uuid
import yaml
import requests
import io

In [2]:
token = '424bdd5a-df56-430b-937a-9e48957cea99'
file = '/home/sdanioth/Documents/git/oscar-catalogue/oscar-observation-catalogue/0-756-1-4802340_531_.xml'

oscar_api = 'https://oscardepl.wmo.int/surface/rest/api/wmd/upload'


with open(file) as r:
    xml = r.read()
    res = requests.post(url=oscar_api, data=xml, headers={'X-WMO-WMDR-Token': token})
    print(res)

<Response [200]>


In [15]:
from pyoscar import OSCARClient
token = '424bdd5a-df56-430b-937a-9e48957cea99'
file = '/home/sdanioth/Documents/git/oscar-catalogue/oscar-observation-catalogue/0-756-1-4802340_531_.xml'


client = OSCARClient(api_token=token)
print(client)

with open(file) as fh:
    data = fh.read()

response = client.upload(data)
print(response)

{'id': 689546, 'xmlStatus': 'VALID_XML_WITH_ERRORS_OR_WARNINGS', 'logs': 'The list below is organized by section header and shows exceptions/issues – if any – that may have resulted from the processing of the XML (NB: Section headers are always displayed).\n# Facility with identifier "0-756-1-4802340"\nREF_6: The selected identification process is using the ID attribute. No value found for the ID attribute of the element "deployment/Deployment" with beginPosition = "undefined". The "deployment" is discarded.\n'}


In [4]:
class FacilityCatalogue:
    """
    Read flat file, convert to WMDR XML file, upload to OSCAR/Surface.

    Read flat file containing facility catalogue entries, 
    convert to WMDR XML file, upload to OSCAR/Surface.
    """

    @classmethod
    def __init__(self, config):
        """
        Constructor

        Parameters
        ----------
        config : str
            Full path to configuration file.

        Returns
        -------
        None.

        """
        try:                        
            with open(os.path.abspath(config), "r") as f:
                config = yaml.safe_load(f)
                f.close()

            # setup logging
            for h in logging.root.handlers[:]:
                logging.root.removeHandler(h)
            logdir = os.path.expanduser(config['logging'])
            os.makedirs(logdir, exist_ok=True)
            logfile = '%s.log' % time.strftime('%Y%m%d')
            self.logfile = os.path.join(logdir, logfile)
            self.logger = logging.getLogger(__name__)
            logging.basicConfig(level=logging.DEBUG,
                                format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                                datefmt='%y-%m-%d %H:%M:%S',
                                filename=str(self.logfile),
                                filemode='a')
            
            self.source = os.path.expanduser(config['source'])
            self.target = os.path.expanduser(config['target'])
            self.template = config['template'] 
            self.header = { 'dtm': time.strftime("%Y-%m-%dT%H:%M:%S"), 
                    'individualName': config['individualName'],
                            'organisationName': config['organisationName'],
                            'electronicMailAddress': config['organisationName'] }

            self.upload = config['upload']
            self.proxies = config['proxies']
            self.token = config['token']
            
        except Exception as err:
            self.logger.error('Error setting up FacilityCatalogue', err)



    @classmethod
    def csv2wmdr(self, verbose=True):
        """
        Read a flat CSV file, cast to WMDR XML file, save as target

        Parameters
        ----------
        verbose : bln, optional
            Should log file be written? The default is True.

        Returns
        -------
        filename with full path.

        """
        try:
            sanitize = lambda x: html.escape(x) if len(x) > 0 else 'unknown'

            # read jinja template
            templateLoader = jinja2.FileSystemLoader(searchpath="./")
            templateEnv = jinja2.Environment(loader=templateLoader)
            template = templateEnv.get_template(self.template)

            # read source file and convert to list of dictionaries, one for
            # each combination of manufacturer, model, variable
            with open(self.source) as f:
                reader = csv.DictReader( f )
                files = []                
                for row in reader:
                    var = 'ObservedVariableAtmosphere'
                    d1 = { key: sanitize(row[key]) for key in row.keys() if key != var }
                    for ele in row[var].split(','):
                        facility = {var: ele.strip(), 'uuid': uuid.uuid4()} 
                        facility.update(d1)
                                   
                        # generate XML file
                        xml = template.render(header=self.header, facility=facility)
                        file = os.path.join(self.target, "%s %s %s.xml" % (facility['Name'], facility['Territory'], facility['ObservedVariableAtmosphere']))
                        file = file.replace(" ", "_")
                        files.append(file)                    
                        with open(file, 'w') as f:
                            f.write(xml)
                            f.close()
                        if verbose:
                            self.logger.info("XML file saved to " + file)
                
            return(files)

        except Exception as err:
            self.logger.error(err)
            return(err)
        
        

if __name__ == '__main__':
    config = os.path.join(os.getcwd(), 'config.yaml')
    facility_catalogue = FacilityCatalogue(config)
    facility_catalogue.csv2wmdr()
    

In [15]:
config = os.path.join(os.getcwd(), 'config.yaml')

with open(os.path.abspath(config), "r") as f:
    config = yaml.safe_load(f)
    f.close()

source = os.path.expanduser(config['source'])
target = os.path.expanduser(config['target'])
template = config['template'] 
header = { 'dtm': time.strftime("%Y-%m-%dT%H:%M:%S"),'individualName': config['individualName'],'organisationName': config['organisationName'],'electronicMailAddress': config['organisationName'] }

upload = config['upload']
proxies = config['proxies']
token = config['token']
